In [13]:
import argparse
import glob
from pathlib import Path

import open3d
from visual_utils import open3d_vis_utils as V

import os
import numpy as np
import argparse
import json
import sys
import numba
import yaml
import multiprocessing
import shutil
import cv2
import torch

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

# 3DMOT module
from mot_3d.mot import MOTModel
from mot_3d.frame_data import FrameData

import time


In [14]:
class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, ext='.bin'):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.root_path = root_path
        self.ext = ext
        data_file_list = glob.glob(
            str(root_path / f'*{self.ext}')) if self.root_path.is_dir() else [self.root_path]

        data_file_list.sort()
        self.sample_file_list = data_file_list

    def __len__(self):
        return len(self.sample_file_list)

    def __getitem__(self, index):
        points = np.fromfile(
            self.sample_file_list[index], dtype=np.float32, count=-1).reshape([-1, 5])[:, :4]

        sweep_points_list = [points]
        sweep_times_list = [np.zeros((points.shape[0], 1))]

        points = np.concatenate(sweep_points_list, axis=0)
        times = np.concatenate(sweep_times_list, axis=0).astype(points.dtype)

        points = np.concatenate((points, times), axis=1)

        input_dict = {
            'points': points,
            'frame_id': times,
        }

        data_dict = self.prepare_data(data_dict=input_dict)
        return data_dict

In [15]:
# def parse_config():
#     cfg_file = "cfgs/nuscenes_models/cbgs_voxel01_res3d_centerpoint.yaml"
#     ckpt = "cfgs/ckpt/nuScenes/cbgs_voxel01_centerpoint_nds_6454.pth"
#     data_path = "/home/charles/myDataSet/nuScenes/v1.0-mini/sweeps/LIDAR_TOP/"
#     # data_path='/home/charles/myDataSet/nuScenes/v1.0-mini/samples/LIDAR_TOP/'

#     parser = argparse.ArgumentParser(description='arg parser')
#     parser.add_argument('--cfg_file', type=str, default=cfg_file,
#                         help='specify the config for demo')
#     parser.add_argument('--data_path', type=str, default=data_path,
#                         help='specify the point cloud data file or directory')
#     parser.add_argument('--ckpt', type=str, default=ckpt,
#                         help='specify the pretrained model')
#     parser.add_argument('--ext', type=str, default='.bin',
#                         help='specify the extension of your point cloud data file')

#     args = parser.parse_args()

#     cfg_from_yaml_file(args.cfg_file, cfg)

#     return args, cfg

In [16]:
def nuScenes_MOT(tracker, input_data: FrameData):
    results = tracker.frame_mot(input_data)
    return results

In [17]:
logger = common_utils.create_logger()
logger.info(
    '-----------------Quick nuScenes Demo of OpenPCDet-------------------------')

2022-06-22 22:33:38,075   INFO  -----------------Quick nuScenes Demo of OpenPCDet-------------------------
2022-06-22 22:33:38,075   INFO  -----------------Quick nuScenes Demo of OpenPCDet-------------------------


In [20]:
# cfg_file = "cfgs/nuscenes_models/cbgs_voxel01_res3d_centerpoint.yaml"
# ckpt = "cfgs/ckpt/nuScenes/cbgs_voxel01_centerpoint_nds_6454.pth"

cfg_file = "cfgs/nuscenes_models/cbgs_pp_multihead.yaml"
ckpt = "cfgs/ckpt/nuScenes/pp_multihead_nds5823_updated.pth"

data_path = "/home/charles/myDataSet/nuScenes/v1.0-mini/sweeps/LIDAR_TOP/"
cfg = cfg_from_yaml_file(cfg_file, cfg)

# args, cfg = parse_config()

In [21]:
# 加载数据集
demo_dataset = DemoDataset(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    training=False,
    root_path=Path(data_path),  logger=logger
)

logger.info(f'Total number of samples: \t{len(demo_dataset)}')

# 加载模型
model = build_network(model_cfg=cfg.MODEL, num_class=len(
    cfg.CLASS_NAMES), dataset=demo_dataset)

# 导入模型数据
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False)
model.cuda()
model.eval()


2022-06-22 22:33:59,130   INFO  Total number of samples: 	3531
2022-06-22 22:33:59,130   INFO  Total number of samples: 	3531
/home/charles/anaconda3/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2022-06-22 22:34:01,694   INFO  ==> Loading parameters from checkpoint cfgs/ckpt/nuScenes/pp_multihead_nds5823_updated.pth to GPU
2022-06-22 22:34:01,694   INFO  ==> Loading parameters from checkpoint cfgs/ckpt/nuScenes/pp_multihead_nds5823_updated.pth to GPU
2022-06-22 22:34:01,734   INFO  ==> Done (loaded 421/421)
2022-06-22 22:34:01,734   INFO  ==> Done (loaded 421/421)


cfgs/ckpt/nuScenes/pp_multihead_nds5823_updated.pth


PointPillar(
  (vfe): PillarVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): Linear(in_features=11, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (backbone_3d): None
  (map_to_bev_module): PointPillarScatter()
  (pfe): None
  (backbone_2d): BaseBEVBackbone(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (8): BatchNorm2d(64

In [22]:
# 3D界面初始化
vis = open3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().point_size = 1.0
vis.get_render_option().background_color = np.zeros(3)

# 跟踪器初始化
# load model configs
config_path = '../3DMOT_configs/nuScenes_cfg/giou.yaml'
configs = yaml.load(open(config_path, 'r'), Loader=yaml.Loader)
tracker = MOTModel(configs)

# 加载摄像头数据
img_path = "/home/charles/myDataSet/nuScenes/v1.0-mini/samples/CAM_FRONT/"
img_file_list = glob.glob(str(Path(img_path) / f'*.jpg'))
img_file_list.sort()

with torch.no_grad():
    for idx, data_dict in enumerate(demo_dataset):
        millis = int(round(time.time() * 1000))
        logger.info(f'Visualized sample index: \t{idx + 1}')
        data_dict = demo_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
        pred_dicts, _ = model.forward(data_dict)

        # # CAMERA显示
        # image = cv2.imread(img_file_list[idx])
        # cv2.namedWindow("CAM FRONT",0)
        # cv2.resizeWindow("CAM FRONT", 640, 480)
        # cv2.imshow('CAM FRONT',image)
        # cv2.waitKey(1)

        # 原始点云和检测显示
        V.draw_scenes(vis,
                        points=data_dict['points'][:, 1:],
                        ref_boxes=pred_dicts[0]['pred_boxes'],
                        ref_scores=pred_dicts[0]['pred_scores'],
                        ref_labels=pred_dicts[0]['pred_labels']
                        )

2022-06-22 22:34:05,371   INFO  Visualized sample index: 	1
2022-06-22 22:34:05,371   INFO  Visualized sample index: 	1
2022-06-22 22:34:05,501   INFO  Visualized sample index: 	2
2022-06-22 22:34:05,501   INFO  Visualized sample index: 	2
2022-06-22 22:34:05,598   INFO  Visualized sample index: 	3
2022-06-22 22:34:05,598   INFO  Visualized sample index: 	3
2022-06-22 22:34:05,703   INFO  Visualized sample index: 	4
2022-06-22 22:34:05,703   INFO  Visualized sample index: 	4


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to

2022-06-22 22:34:05,824   INFO  Visualized sample index: 	5
2022-06-22 22:34:05,824   INFO  Visualized sample index: 	5
2022-06-22 22:34:05,915   INFO  Visualized sample index: 	6
2022-06-22 22:34:05,915   INFO  Visualized sample index: 	6


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to

2022-06-22 22:34:06,007   INFO  Visualized sample index: 	7
2022-06-22 22:34:06,007   INFO  Visualized sample index: 	7
2022-06-22 22:34:06,102   INFO  Visualized sample index: 	8
2022-06-22 22:34:06,102   INFO  Visualized sample index: 	8
2022-06-22 22:34:06,193   INFO  Visualized sample index: 	9
2022-06-22 22:34:06,193   INFO  Visualized sample index: 	9


...

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to

2022-06-22 22:34:19,895   INFO  Visualized sample index: 	160
2022-06-22 22:34:19,895   INFO  Visualized sample index: 	160
2022-06-22 22:34:19,980   INFO  Visualized sample index: 	161
2022-06-22 22:34:19,980   INFO  Visualized sample index: 	161
2022-06-22 22:34:20,066   INFO  Visualized sample index: 	162
2022-06-22 22:34:20,066   INFO  Visualized sample index: 	162


[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to

2022-06-22 22:34:20,154   INFO  Visualized sample index: 	163
2022-06-22 22:34:20,154   INFO  Visualized sample index: 	163
2022-06-22 22:34:20,244   INFO  Visualized sample index: 	164
2022-06-22 22:34:20,244   INFO  Visualized sample index: 	164
2022-06-22 22:34:20,330   INFO  Visualized sample index: 	165
2022-06-22 22:34:20,330   INFO  Visualized sample index: 	165
2022-06-22 22:34:20,416   INFO  Visualized sample index: 	166
2022-06-22 22:34:20,416   INFO  Visualized sample index: 	166
2022-06-22 22:34:20,500   INFO  Visualized sample index: 	167
2022-06-22 22:34:20,500   INFO  Visualized sample index: 	167
2022-06-22 22:34:20,585   INFO  Visualized sample index: 	168
2022-06-22 22:34:20,585   INFO  Visualized sample index: 	168
2022-06-22 22:34:20,674   INFO  Visualized sample index: 	169
2022-06-22 22:34:20,674   INFO  Visualized sample index: 	169
2022-06-22 22:34:20,763   INFO  Visualized sample index: 	170
2022-06-22 22:34:20,763   INFO  Visualized sample index: 	170
2022-06-

KeyboardInterrupt: 